<a href="https://colab.research.google.com/github/sayantan11995/wikipedia_enrichment/blob/main/Finetuning/finetunellama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb==0.4.14 sentence-transformers wikipedia datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build w

In [ ]:
## Chromadb issue
# !pip install --upgrade chromadb==0.4.14

In [2]:
import requests
import re
import wikipedia
import pandas as pd
import shutil
import time
from tqdm import tqdm
from langchain.embeddings import HuggingFaceEmbeddings
from ast import literal_eval
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [4]:
## Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/wikipedia_fa_biographical_data_RAG.csv')
data = data.head(2000)



In [ ]:
#######################
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separators=["\n\n", "\n", " ", "."],)
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
#######################


for idx, rows in tqdm(data.iterrows()):

    external_content = ""

    links = literal_eval(rows['wiki_links'])
    for external_page in links:
        try:
            external_content += "\n" + wikipedia.page(external_page, auto_suggest=False).content

            # Remove section names using regex (=== Name ===)
            external_content = re.sub(r'==.*?==', '', external_content)
            external_content = re.sub(r'\n\n', '\n', external_content)

            # Remove non alpha numeric characters from text
            external_content = re.sub(r'[^a-zA-Z0-9\s]', '', external_content)
        except:
            pass


    all_splits = text_splitter.split_text(external_content)
    shutil.rmtree("chroma_db", ignore_errors=True)
    # time.sleep(1)
    print(len(all_splits))
    if len(all_splits) == 0:
        data.loc[idx, 'context'] = None
        continue
    vectordb = Chroma.from_texts(texts=all_splits, embedding=embeddings, persist_directory="chroma_db")
    # vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

    # Retriving Top n Chunks most Similar to query.
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})

    ## Retriving relevant context for the last section
    context = "\n\n".join([doc.page_content for doc in retriever.get_relevant_documents(rows['last_paragraph'])])

    # print(len(context))
    data.loc[idx, 'context'] = context

    # break


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
0it [00:00, ?it/s]

277


1it [00:09,  9.58s/it]

390


2it [00:27, 14.22s/it]

...

1998it [8:16:39,  7.98s/it]

347


1999it [8:16:49,  8.70s/it]

365


2000it [8:17:00, 14.91s/it]


In [4]:
import pandas as pd
# data.to_csv('/content/drive/MyDrive/Colab Notebooks/wikipedia_enrichment/data/wiki_training_data_with_context.csv', index=False)

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/wikipedia_enrichment/data/wiki_training_data_with_context.csv')

In [ ]:
data.shape

(2000, 6)

In [7]:
import datasets
# ds = datasets.load_dataset("imdb", split="train")

print(ds[0]['text'])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [8]:
import datasets
temp_dataset = datasets.Dataset.from_pandas(data)

prompt_template = "<s>[INST] Generate a coherent paragraph relevant to the given section content. \n\n Section Content: {}: {} \n\n Context: {} [/INST] {} </s>"
temp_dataset = temp_dataset.map(lambda x: {"text": prompt_template.format(x["section"], x["incomplete_content"], x["context"], x["last_paragraph"])})

# Remove every other column except text
temp_dataset = temp_dataset.remove_columns(["incomplete_content", "context", "last_paragraph", 'title', 'section', 'wiki_links'])


print(temp_dataset[0]["text"])



Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<s>[INST] Generate a coherent paragraph relevant to the given section content. 

 Section Content: Early_life,_family_and_education: Aaliyah Dana Haughton was born on January 16, 1979, in Brooklyn, New York City,  the younger child of Diane and Michael "Miguel" Haughton, a warehouse worker. She was of African-American descent. Her name is the feminine form of the Arabic "Ali", meaning "highest, most exalted one, the best." Aaliyah was fond of her name, calling it "beautiful" and saying she was "very proud of it" and strove to live up to her name every day. When she was five years old, her family moved to Detroit, Michigan, where she was raised along with her older brother, Rashad. In Detroit, her father began working in the warehouse business, one of his brother-in-law Barry Hankerson's widening interests. Her mother stayed home and raised her and her brother. Her mother enrolled her in voice lessons at an early age. Eventually, she started performing at weddings, church choir, and cha

In [10]:
import torch; torch.version.cuda


'12.1'

In [12]:
torch.__version__

'2.2.1+cu121'

In [1]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-_15z9dig/unsloth_244794ac9a544942a38d0ebaf891598a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-_15z9dig/unsloth_244794ac9a544942a38d0ebaf891598a
  Resolved https://github.com/unslothai/unsloth.git to commit 4211cc01409e3ced4f7abebaf68e244193b46e2c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
## Install proper xformers
## See compatibility: https://anaconda.org/xformers/xformers/files
!pip install xformers==0.0.25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Data Preparation

In [6]:
import pandas as pd
import datasets
# data.to_csv('/content/drive/MyDrive/Colab Notebooks/wikipedia_enrichment/data/wiki_training_data_with_context.csv', index=False)

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/wikipedia_enrichment/data/wiki_training_data_with_context.csv')

dataset = datasets.Dataset.from_pandas(data)

In [ ]:
# prompt_template = "<s>[INST] Generate a coherent paragraph relevant to the given section content. \n\n Section Content: {}: {} \n\n Context: {} [/INST] {} </s>"
# temp_dataset = temp_dataset.map(lambda x: {"text": prompt_template.format(x["section"], x["incomplete_content"], x["context"], x["last_paragraph"])})

# # Remove every other column except text
# temp_dataset = temp_dataset.remove_columns(["incomplete_content", "context", "last_paragraph", 'title', 'section', 'wiki_links'])


# print(temp_dataset[0]["text"])


In [7]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
As a wikipedia writing assistant, generate a coherent paragraph relevant to the given section content from the external content.

### Input:
Section Content: {}: {}

### Exteral Content:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    texts = []
    for title, incomplete_content, context, output in zip(examples["section"], examples["incomplete_content"], examples["context"], examples["last_paragraph"]):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(title, incomplete_content, context, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # max_steps = 6,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [19]:
import torch; torch.cuda.empty_cache()

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 186
 "-____-"     Number of trainable parameters = 39,976,960


Step,Training Loss
10,1.983400
20,1.834800
30,1.944700
40,2.774500
50,2.890300
60,13.196700
70,15.855900
80,7.722700
90,18.932000
100,16.951100


In [11]:
save_path = "/content/drive/MyDrive/Colab Notebooks/wikipedia_enrichment/saved_models/finetuned_llama-2-7b-wikipedia-section-completion"
# trainer.model.save_pretrained(save_path)
model.save_pretrained(save_path) # Local saving

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
test_data = dataset[600]
test_data

{'title': 'Anne,_Queen_of_Great_Britain',
 'section': 'Marriage',
 'incomplete_content': "In November 1677, Anne's sister, Mary, married their Dutch first cousin William III of Orange at St James's Palace, but Anne could not attend the wedding because she was confined to her room with smallpox. By the time she recovered, Mary had already left for her new life in the Netherlands. Lady Frances Villiers contracted the disease and died. Anne's aunt Lady Henrietta Hyde (the wife of Laurence Hyde) was appointed as her new governess. A year later, Anne and her stepmother visited Mary in Holland for two weeks.\n The Duke and Duchess of York retired to Brussels in March 1679 in the wake of anti-Catholic hysteria fed by the Popish Plot, and Anne visited them from the end of August. In October, all three returned to Britain: Anne to England, and her father and stepmother to Scotland. She joined them at Holyrood Palace in Edinburgh from July 1681 until May 1682. It was her last journey outside Eng

In [14]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = save_path, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        test_data["section"],
        test_data["incomplete_content"],
        test_data["context"],
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
output = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
As a wikipedia writing assistant, generate a coherent paragraph relevant to the given section content from the external content.

### Input:
Section Content: Marriage: In November 1677, Anne's sister, Mary, married their Dutch first cousin William III of Orange at St James's Palace, but Anne could not attend the wedding because she was confined to her room with smallpox. By the time she recovered, Mary had already left for her new life in the Netherlands. Lady Frances Villiers contracted the disease and died. Anne's aunt Lady Henrietta Hyde (the wife of Laurence Hyde) was appointed as her new governess. A year later, Anne and her stepmother visited Mary in Holland for two weeks.
 The Duke and Duchess of York retired to Brussels in March 1679 in the wake of anti-Catholic hysteria fed by the Popish Plot, and Ann

In [30]:
# Extract the geerated content from the output
output_text = tokenizer.decode(output[0], skip_special_tokens = True)
print(output_text)


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
As a wikipedia writing assistant, generate a coherent paragraph relevant to the given section content from the external content.

### Input:
Section Content: Marriage: In November 1677, Anne's sister, Mary, married their Dutch first cousin William III of Orange at St James's Palace, but Anne could not attend the wedding because she was confined to her room with smallpox. By the time she recovered, Mary had already left for her new life in the Netherlands. Lady Frances Villiers contracted the disease and died. Anne's aunt Lady Henrietta Hyde (the wife of Laurence Hyde) was appointed as her new governess. A year later, Anne and her stepmother visited Mary in Holland for two weeks.
 The Duke and Duchess of York retired to Brussels in March 1679 in the wake of anti-Catholic hysteria fed by the Popish Plot, and Anne v

In [ ]:
# Dataset
# data_name = "mlabonne/guanaco-llama2-1k"
# training_data = load_dataset(data_name, split="train")

training_data = temp_dataset

# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "/content/drive/MyDrive/Colab Notebooks/wikipedia_enrichment/saved_models/llama-2-7b-wikipedia" #You can give it your own name

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [ ]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

print("abcd")
# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=25,
    learning_rate=2e-5,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params,
    max_seq_length=512
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

abcd


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 89.06 MiB is free. Process 49536 has 14.66 GiB memory in use. Of the allocated memory 14.21 GiB is allocated by PyTorch, and 328.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Generate Text
query = "How do I use the OpenAI API?"
text_gen = pipeline(task="text-generation", model=refined_model, tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"<s>[INST] {query} [/INST]")
print(output[0]['generated_text'])

OSError: llama-2-7b-mlabonne-enhanced does not appear to have a file named config.json. Checkout 'https://huggingface.co/llama-2-7b-mlabonne-enhanced/None' for available files.